# 2D Monitoring Element Exploration

This notebooks provides some examples on how to access and visualize 2D monitoring elements using `cmsdials` and the visualization tools provided by `dqmexplore`.

## Imports

In [ ]:
# Setting up DIALS object and importing dqmexplore
import sys
sys.path.append("../src/")

from utils.setupdials import setup_dials_object_deviceauth
dials = setup_dials_object_deviceauth()

import dqmexplore as dqme

In [ ]:
# Importing DIALS objects
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

# Importing other essential packages
import numpy as np
import pandas as pd

In [ ]:
# Run this cell to see the list of all available 2D MEs
mes_df = pd.DataFrame([me_qry_rslt.__dict__ for me_qry_rslt in dials.mes.list()])
avail_mes_df = mes_df[mes_df["dim"] == 2]

metype_names = []
for me in avail_mes_df["me"]:
    metype_name = me.split("/")[0]
    if metype_name not in metype_names: metype_names.append(metype_name)
        
print("Available types of MEs:", metype_names)
for me in avail_mes_df[avail_mes_df["me"].str.contains(metype_names[0])]["me"]:
    print(me)
avail_mes_df

## 2D Monitoring Elements

In [ ]:
# Obtaininng ME data from DIALS
runnb = 380238
me__regex = "PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_(1|2)"

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

# Checking of any empty LSs in the selected MEs
dqme.exploreutils.check_empty_lss(data2D)

In [ ]:
# Getting trigger rate and plotting it
trig_rate = dqme.omsutils.get_rate(dials, runnb, "ZeroBias")
dqme.omsutils.plot_rate(trigratedata, norm=False, show=True)

In [ ]:
# Defining plot features and plotting
ax_labels = [
    dict(x="SignedModuleCoord", y="SignedLadderCoord")
] * 4

fig_title = f"Pixel Barrel Digi Occupancy Normalized by Trigger Rate (Run {runnb})"

fig2D = dqme.plotting.plot2DMEs(
    data2D,
    fig_title=fig_title,
    ax_labels=ax_labels,
    trigger_rates=trig_rate,
    height=800,
    width=1250,
    hspace=0.2,
    show=True
)

If you wish to further modify the figure `dqmexplore` gives you, you can do so by setting `show=False`, and using `plotly` to change the elements of the plot.

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add modifications to fig2D here.


And if you wish to export the plot to an HTML file, you can run the code show below after setting `show=False`.

In [ ]:
# You can set show=False and export the figure to an HTML file by also running the code below
from plotly.offline import plot
plot(fig2D, filename=f"./fig2d.html")